In [ ]:
import sys
sys.path.append('./src')
import syllo_gen
import random
import json
from tqdm.autonotebook import tqdm
import openai
openai.api_key = open('key.txt').read().strip()

%load_ext autoreload
%autoreload 2

In [ ]:
# import gpt_comp
# gpt_comp.test_composition(dmax=5, shuffle_story=False)

# ADJ Template

In [ ]:
q,v = syllo_gen.get_syllo(3)
# q = syllo_gen.negate_quesion(q)
v = syllo_gen.random_assign_adjs(v)
print(syllo_gen.question2fol(q, v))
print(syllo_gen.question2template(q, v, rand=True, noun=False))

# NOUN Template

In [ ]:
q,v = syllo_gen.get_syllo(1)
# q = syllo_gen.negate_quesion(q)
v = syllo_gen.random_assign_nouns(v)  # change to correct type
print(syllo_gen.question2fol(q, v))
print(syllo_gen.question2template(q, v, rand=True, noun=True)) # change to correct type


In [ ]:
all_possilbe = set()
# prompt = 'Answer questions about syllogisms, ignoring semantics.\n\n'
prompt = 'Answer questions about syllogisms.\n\n'
# prompt = ''
rand = False
noun = False
for j in range(100):
    if len(all_possilbe) == 24:
        break
    real = random.choice([True, False])
    q, v = syllo_gen.get_syllo(1)

    qs = syllo_gen.question_to_string(q)
    if qs in all_possilbe:
        continue
    all_possilbe.add(qs)

    if not real:
        q = syllo_gen.negate_quesion(q)

    # if random.random() < 0.5:
    #     v = syllo_gen.assign_greek_letters(v)

        
    if noun:
        v = syllo_gen.random_assign_nouns(v)
    else:
        v = syllo_gen.random_assign_adjs(v)

    
    q = syllo_gen.question2template(q, v, rand=rand, noun=noun)
    # q = syllo_gen.question2fol(q, v)

    prompt += f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}\n'
    prompt += f'Answer: {real}\n\n'

# for j in range(100):
#     if len(all_possilbe) == 30:
#         break
#     real = random.choice([True, False])
#     q, v = syllo_gen.get_syllo(2)
#     if not real:
#         q = syllo_gen.negate_quesion(q)

#     v = syllo_gen.random_assign_nouns(v)
#     if tuple(q['story']) not in all_possilbe:
#         all_possilbe.add((tuple(q['story'])))
#         q = syllo_gen.question2template(q, v, rand=rand)
#         prompt += f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}\n'
#         prompt += f'Answer: {real}\n\n'
print(prompt)

In [ ]:
corret = 0
total= 1000
evaled = 0
all_possilbe = set()

for i in tqdm(range(total)):

    # if len(all_possilbe) == 48:
    #     break

    real = random.choice([True, False])
    q, v = syllo_gen.get_syllo(1)
    if not real:
        q = syllo_gen.negate_quesion(q)

    # qs = syllo_gen.question_to_string(q)
    # if qs not in all_possilbe:
    #     all_possilbe.add(qs)
    # else:
    #     continue

    # if random.random() < 1:
    #     v = syllo_gen.assign_greek_letters(v)

    if noun:
        v = syllo_gen.random_assign_nouns(v)
    else:
        v = syllo_gen.random_assign_adjs(v)

    q = syllo_gen.question2template(q, v, rand=rand, noun=noun)
    # q = syllo_gen.question2fol(q, v)

    tmp_prompt = prompt + f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}'

    tmp_prompt += "\nAnswer:" 

    # print(tmp_prompt)
    # break


    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=tmp_prompt,
    temperature=0,
    max_tokens=10,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"]
    )
    text = response['choices'][0]['text'].strip()
    if text not in ['True', 'False']:
        print('Error')
        print(text)
        continue
    if text == 'True' and real:
        corret += 1
    elif text == 'False' and not real:
        corret += 1
    else:
        print(f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}')
        print(text)


print(corret/total)